In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight


In [2]:
BASE_DIR = r"C:\Projects\Dyspraxia\Dyspraxia_Dataset"

TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR   = os.path.join(BASE_DIR, "val")
TEST_DIR  = os.path.join(BASE_DIR, "test")



In [3]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 64

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)



In [5]:
train_gen = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)


Found 73500 images belonging to 3 classes.
Found 21000 images belonging to 3 classes.
Found 10500 images belonging to 3 classes.


In [12]:
# Use only a subset of training data (e.g., 50%)
SUBSET_FRACTION = 0.3

steps_per_epoch = int(len(train_gen) * SUBSET_FRACTION)
validation_steps = len(val_gen)

print("Steps per epoch (subset):", steps_per_epoch)


Steps per epoch (subset): 344


In [13]:
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_gen.classes),
    y=train_gen.classes
)

class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)



Class weights: {0: np.float64(1.0), 1: np.float64(1.0), 2: np.float64(1.0)}


In [14]:
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(128, 128, 3)
)

base_model.trainable = False  # BASELINE



In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 128, 128,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 128, 128,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 128, 128,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 129, 129,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 64, 64,    │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 64, 64,    │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 64, 64,    │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 64, 64,    │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 64, 64,    │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 64, 64,    │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 64, 64,    │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 64, 64,    │        512 │ block1a_se_excit

 Total params: 4,219,046 (16.09 MB)

 Trainable params: 166,915 (652.01 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

In [16]:
callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

history = model.fit(
    train_gen,
    steps_per_epoch=steps_per_epoch,   # 🔥 NEW
    validation_data=val_gen,
    validation_steps=validation_steps,
    epochs=10,
    class_weight=class_weights,
    callbacks=callbacks
)




Epoch 1/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 284s 804ms/step - accuracy: 0.5833 - loss: 0.8638 - val_accuracy: 0.7134 - val_loss: 0.9287
Epoch 2/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 279s 812ms/step - accuracy: 0.6751 - loss: 0.6602 - val_accuracy: 0.7803 - val_loss: 0.6501
Epoch 3/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 270s 787ms/step - accuracy: 0.7041 - loss: 0.6089 - val_accuracy: 0.8027 - val_loss: 0.5180
Epoch 4/10
117/344 ━━━━━━━━━━━━━━━━━━━━ 1:29 392ms/step - accuracy: 0.7229 - loss: 0.5760

c:\Projects\Dyspraxia\venv\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


344/344 ━━━━━━━━━━━━━━━━━━━━ 176s 511ms/step - accuracy: 0.7183 - loss: 0.5814 - val_accuracy: 0.7948 - val_loss: 0.5207
Epoch 5/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 289s 841ms/step - accuracy: 0.7217 - loss: 0.5791 - val_accuracy: 0.7900 - val_loss: 0.5193
Epoch 6/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 300s 874ms/step - accuracy: 0.7355 - loss: 0.5514 - val_accuracy: 0.7846 - val_loss: 0.4915
Epoch 7/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 285s 829ms/step - accuracy: 0.7432 - loss: 0.5383 - val_accuracy: 0.7890 - val_loss: 0.5123
Epoch 8/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 158s 458ms/step - accuracy: 0.7525 - loss: 0.5284 - val_accuracy: 0.7657 - val_loss: 0.4920
Epoch 9/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 830s 2s/step - accuracy: 0.7437 - loss: 0.5335 - val_accuracy: 0.7484 - val_loss: 0.5148
Epoch 10/10
344/344 ━━━━━━━━━━━━━━━━━━━━ 312s 907ms/step - accuracy: 0.7522 - loss: 0.5228 - val_accuracy: 0.7500 - val_loss: 0.5156
